## **Agata Semerjak, 402426**  
gr 1a, środa 14:45  
**lab 6** 14.04.2021

**Zadanie 1.**  
  
Algorytm G-TSP

In [18]:
def print_matrix(M):
    print("%3s" % ' ', end='')
    print("%3s" % ' | ', end='')
    for i in range(len(M)):
        print("%7s" % str(i), end='')
    print("\n")
    print("-"*(7*len(M) + 6))
    for r in range(len(M)):
        print("%3s" % str(r), end='')
        print("%3s" % ' | ', end='')
        for c in range(len(M)):
            print("%7s" % str(M[r][c]), end='')
        print("\n")

In [19]:
import numpy as np

def is_cyclic(G):
    visited = {u : False for u in G} #słownik odwiedzonych wierzchołków
    cycle = [False] #zmienna przechowująca informację o znalezeiniu cyklu (w postaci tablicy po to by można było ją przekazać jako parametr do funkcji i tam też zmodyfikować)
    for u in G:
        if not visited[u]:
            DFS(G, u, cycle, u, visited) #DFS dla każdego wierzchołka zatrzymujące się w momencie znalezienia cyklu
        if cycle[0]:
            break
    return cycle[0]

def DFS(G, u, cycle, prev, visited):
    if cycle[0]:
        return #zatrzymanie gdy cykl istnieje
    visited[u] = True #oznaczenie bieżącego wierzchołka jako odwiedzony
    for v in G[u]: #dla każdego sąsiada u przejście DFS i przerwanie w momencie znalezienia cyklu (czyli gdy dotrzemy do już odwiedzonego wierzchołka)
        if visited[v] and v != prev:
            cycle[0] = True
            return
        if not visited[v]:
            DFS(G, v, cycle, u, visited)


def G_TSP(G):
    #G to macierz wag
    E = [] #zbiór krawędzi grafu
    route_graph = {}
    def myFunc(e):
        start, stop, weight = e
        return e #funkcja zwracająca wagę krawędzi (potrzebna do sortowania)

    for i in range(len(G)):
        route_graph[i] = []
        for j in range(len(G)):
            if G[i][j] is not None and i>=j:
                E.append((i, j, G[i][j])) #uzupełnianie krawędzi
                
    E.sort(key=myFunc) #sortowania krawędzi wg rosnących wag
    route = [] #lista krawędzi rozwiązania
    suma = 0 # suma wag rozwiązania
    visited = [] #lista odwiedzonych wierzchołków

    while len(route) != len(G) and E != []: #dopóki droga nie będzei zawierała wszystkich wymaganych krawędzi
        start, stop, weight = E[0]
        E.pop(0)
        if ((visited.count(start)<2 and visited.count(stop)<2)): #jeśli dodanie krawędzi nie spowoduje dodania węzła, z którego wychodzą 3 krawędzie
            route_graph[start].append(stop) #uzupełnianie nowego grafu zawierającego jedynie rozwiązanie (potrzebne do detekcji cyklu)
            route_graph[start].sort()
            route_graph[stop].append(start)
            route_graph[stop].sort()
            if len(route) < len(G) - 1: #jeśli nie jest to ostatnia iteracja
                if is_cyclic(route_graph): #sprawdzenie czy dodanie krawędzi stworzyło cykl (za pomocą DFS)
                    route_graph[start].remove(stop) #usuwanie z grafu jeśli powstał cykl
                    route_graph[stop].remove(start)
                    continue #wyjście z danego obiegu pętli
                
            route.append((start, stop)) #dodanie krawędzi do rozwiązania
            visited.append(start) #uzupełnienie odwiedzonych węzłów
            visited.append(stop)
            suma += weight #uzupełnienie sumarycznej wagi

    return route, suma #zwraca listę krawędzi oraz sumę wag rozwiązania

def print_solution(route, suma): #funkcja zmieniająca listę krawędzi na ścieżkę
    route_list = []
    Q = route
    u, v = Q[0]
    route_list.append(u)
    route_list.append(v)
    Q.pop(0)
    while Q != []:
        for u_, v_ in Q:
            if u_ == v or u_ == u:
                route_list.append(v_)
                u = u_
                v = v_
                Q.remove((u_, v_))
                
            elif v_ == u or v_ == v:
                route_list.append(u_)
                u = u_
                v = v_
                Q.remove((u_, v_))
    print("droga: ", route_list)
    print("suma wag: ", suma)


**Zadanie 2.**
  
Z punktu widzenia algorytmu nie jest istotny wierzchołek startowy, gdyż pracuje on na samych krawędziach. Niezależnie od wierzchołka zawsze zwróci on tę samą ścieżkę. Algorytm został zaimplementowany dla grafów nieskierowanych pełnych.

In [20]:
G1 = [[None, 10, 15, 20], [10, None, 35, 25], [15, 35, None, 30], [20, 25, 30, None]]
print_matrix(G1)
route, suma = G_TSP(G1)
print(route, suma)
print_solution(route, suma)

    |       0      1      2      3

----------------------------------
  0 |    None     10     15     20

  1 |      10   None     35     25

  2 |      15     35   None     30

  3 |      20     25     30   None

[(1, 0), (2, 0), (3, 1), (3, 2)] 80
droga:  [1, 0, 2, 3, 1]
suma wag:  80


In [21]:
G2 = [[None, 2, 2, 4, 3, 10, 11, 13],
[2, None, 2, 9, 22, 21, 1, 13],
[2, 2, None, 8, 2, 1, 11, 41],
[4, 9, 8, None, 1, 2, 12, 3],
[3, 22, 2, 1, None, 21, 4, 5],
[10, 21, 1, 2, 21, None, 2, 2],
[11, 1, 11, 12, 4, 2, None, 2],
[13, 13, 41, 3, 5, 2, 2, None]]
print_matrix(G2)
route, suma = G_TSP(G2)
print(route, suma)
print_solution(route, suma)

    |       0      1      2      3      4      5      6      7

--------------------------------------------------------------
  0 |    None      2      2      4      3     10     11     13

  1 |       2   None      2      9     22     21      1     13

  2 |       2      2   None      8      2      1     11     41

  3 |       4      9      8   None      1      2     12      3

  4 |       3     22      2      1   None     21      4      5

  5 |      10     21      1      2     21   None      2      2

  6 |      11      1     11     12      4      2   None      2

  7 |      13     13     41      3      5      2      2   None

[(1, 0), (2, 0), (3, 1), (4, 2), (5, 3), (6, 4), (7, 5), (7, 6)] 25
droga:  [1, 0, 2, 4, 6, 7, 5, 3, 1]
suma wag:  25


In [22]:
G3 = [[None, 5, 15, 9, 15, 15, 3,15, 3, 3],
    [5, None, 9, 15, 8, 6, 15, 7, 15, 15],
    [15, 9, None, 9, 4, 15, 5, 3, 15, 2],
    [9, 15, 9, None, 15, 15, 8, 15, 15, 9],
    [15, 8, 4, 15, None, 2, 1, 15, 7, 5],
    [15, 6, 15, 15, 2, None, 6, 15, 15, 15],
    [3, 15, 5, 8, 1, 6, None, 9, 5, 6],
    [15, 7, 3, 15, 15, 15, 9, None, 8, 15],
    [8, 7, 3, 15, 6, 15, 15, 15, None, 15],
    [15, 15, 3, 9, 15, 8, 9, 15, 7, None]]
print_matrix(G3)
route, suma = G_TSP(G3)
print(route, suma)
print_solution(route, suma)

    |       0      1      2      3      4      5      6      7      8      9

----------------------------------------------------------------------------
  0 |    None      5     15      9     15     15      3     15      3      3

  1 |       5   None      9     15      8      6     15      7     15     15

  2 |      15      9   None      9      4     15      5      3     15      2

  3 |       9     15      9   None     15     15      8     15     15      9

  4 |      15      8      4     15   None      2      1     15      7      5

  5 |      15      6     15     15      2   None      6     15     15     15

  6 |       3     15      5      8      1      6   None      9      5      6

  7 |      15      7      3     15     15     15      9   None      8     15

  8 |       8      7      3     15      6     15     15     15   None     15

  9 |      15     15      3      9     15      8      9     15      7   None

[(1, 0), (2, 0), (3, 1), (4, 2), (5, 3), (6, 4), (7, 5), (8, 6), 

Algorytm zwraca poprawne wyniki dla wszystkich przyjętych przykładów.

**Zadanie 3.**
  
Główna pętla algorytmu wykona się maksymalnie n^2 razy (n - liczba krawędzi). W każdej iteracji następuje sprawdzenie, czy daną krawędź można dołączyć do rozwiązania. Złożoność obliczeniowa algorytmu zależy od sposobu sprawdzania tego warunku, a także od sposobu sortowania kolejki krawędzi. Przyjmuje się, że złożoność ta wynosi O(n^2log(n)).  
Najistotniejszą cechą algorytmu G-TSP jest fakt, że jest to algorytm zachłanny pracujący na posortowanych krawędziach, a co za tym idzie - krawędzie nie są dołączane po kolei wzdłuż ścieżki ani nie od konkretnego wierzchołka. Dodatkowe przyjęte rozwiązanie jest gotowe, nie trzeba szukać 'lepszych rozwiązań' w kolejnych krokach (w odróżnieniu od np algorytmu 2-opt).